# You Tube transcripts

In [1]:
import os
import pandas as pd
import openai
from openai import OpenAI
from tqdm import tqdm

from utils import Crawler, Analyzer, aggregate_analysis_files, VideoInfo

# Access the environment variable
yt_api_key = os.getenv('YT_API_KEY')
openai_api_key = os.getenv("OPENAI_API_KEY")

%load_ext autoreload
%autoreload 2

# Search for YT channels of Florida cities and towns

So we have a list of all cities and towns of Florida, let's take a look at it:

Let's crawl for all the cities in the list of cities:

In [2]:
crawler = Crawler()

In [3]:
crawler.start()

0it [00:00, ?it/s]ERROR:tornado.general:SEND Error: Host unreachable
49it [01:44,  2.14s/it]

Quota exceeded. Stopping crawling.


In [8]:
analyzer = Analyzer()

In [9]:
analyzer.start()

437it [01:45,  4.16it/s]


In [ ]:
aggregate_analysis_files('aggregated_analysis.csv')

## Extract info from one video with video id

In [3]:
# Example YouTube video ID
# https://www.youtube.com/watch?v=AH4XSLg6rvw&t=588s
video_id = "5AIBek1jhfE"  # ID is what comes after 'v=' in URL: https://www.youtube.com/watch?v=5AIBek1jhfE

In [5]:
video = VideoInfo(video_id)

In [24]:
import os
from googleapiclient.discovery import build


# Initialize the YouTube Data API client
youtube = build('youtube', 'v3', developerKey=yt_api_key)


def get_channel_id(youtube, channel_username):
    request = youtube.channels().list(
        part="id",
        forUsername=channel_username
    )
    response = request.execute()
    return response['items'][0]['id'] if response['items'] else None


def get_live_streams_from_channel(channel_id):
    videos = []
    next_page_token = None

    while True:
        request = youtube.search().list(
            part="snippet",
            channelId=channel_id,
            maxResults=50,
            pageToken=next_page_token,
            order="date",
            type="video",
            eventType="live"  # This filters for live streams only
        )
        response = request.execute()

        for item in response['items']:
            video_info = {
                "video_id": item['id']['videoId'],
                "title": item['snippet']['title'],
                "description": item['snippet']['description'],
                "published_at": item['snippet']['publishedAt']
            }
            videos.append(video_info)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    return videos


In [ ]:

# Example channel username (can also be channel ID if known)
channel_username = "townoffortmyersbeachfl33931"

# Get the channel ID
channel_id = get_channel_id(youtube, channel_username)
if not channel_id:
    print(f"Channel not found for username: {channel_username}")
else:
    # Get the live streams from the channel
    live_streams = get_live_streams_from_channel(channel_id)
    
    # Print the live streams
    for video in live_streams:
        print(f"Video ID: {video['video_id']}")
        print(f"Title: {video['title']}")
        print(f"Description: {video['description']}")
        print(f"Published At: {video['published_at']}")
        print("-" * 40)